In [1]:
# Load packages and Open data

%pylab
import happi

plt.rc('text', usetex=False)
mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 16#20
# mpl.rcParams['legend.fontname'] = 'Comic Sans MS'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 1.2

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet"
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

Units = happi.Units("mm", "cm^{-3}", "ns", "V/m", "T", "km/s","MeV")

# wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024/'
wkdir = [
        '/Volumes/LaCie/Data_Smilei/SS/SS1_width200um/',
        '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024_2/',
        '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024/',
#         '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz40_angle90_Te60_Ti20_Lx2048_dx02_ppc1024/',
#          '/Volumes/LaCie/Data_Smilei/SS_v1500_BxBz_30deg/',
#          '/Volumes/LaCie/Data_Smilei/SS_v1500_BxBz_60deg/',
#          '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz0_angle90_Te100_Ti200_Lx2048_dx02_ppc1024_2/',
#          '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te600_Ti200_Lx2048_dx02_ppc1024/'
        ]
# wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te600_Ti200_Lx2048_dx02_ppc1024/'


S0 = happi.Open(wkdir[0], reference_angular_frequency_SI=56375055300167.87)
S1 = happi.Open(wkdir[1], reference_angular_frequency_SI=56375055300167.87)
S2 = happi.Open(wkdir[2], reference_angular_frequency_SI=56375055300167.87)

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
Loaded simulation '/Volumes/LaCie/Data_Smilei/SS/SS1_width200um/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Loaded simulation '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024_2/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Loaded simulation '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024/'
Scanning for Sc

In [2]:
# prepare constants, units

me = 9.1e-31
mp = 1836.*me
qe = 1.6e-19
ep = 8.9e-12  # epsilon_0
c  = 3.0e8
wr = S0.namelist.w_r
de = c / wr
Lx = S0.namelist.L_x.real * de * 1e3      # in mm
dx = S0.namelist.d_x * de * 1e3           # in mm

Te = S0.namelist.T_e * 511.e3             # in eV
Ti = S0.namelist.T_i * 511.e3             # in eV
ne = 1.0e18                              # in cm-3
ni = 1.0e18
ld = 7.43e2 * (Te**0.5 * ne**(-0.5) + Ti**0.5 * ni**(-0.5)) * 10. # in mm
dt = S0.namelist.d_t

B0 = S0.namelist.B_z * (me * wr / qe)
wc = qe * B0 / me

print('dx / ld = {:.2f}'.format(dx/ld))

dx / ld = 5.60


In [26]:
plt.rc('text', usetex=False)
mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 16#20
# mpl.rcParams['legend.fontname'] = 'Comic Sans MS'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 1.2

nbal0 = S0.Scalar('Ubal_norm',units=['V/m','um','ns','cm^-3','J'],color='y',label='Bz=20T')
nbal1 = S1.Scalar('Ubal_norm',units=['V/m','um','ns','cm^-3','J'],color='m',label='Bz=10T')
# nbal2 = S2.Scalar('Ubal_norm',units=['V/m','um','ns','cm^-3','J'],color='g',label='Te=300eV, lmd=0.12dx')

happi.multiPlot(
                nbal1,
                nbal0,
#                 nbal2,
                figsize=[3.14*2,3.14*2],
                xmax=2.5
               )
legend()
grid()
nbal1.set(xmax=2.5)
# savefig('/Users/yz/Desktop/Debye_energy_conservation.pdf')

Scalar Ubal_norm


In [3]:
# Function for spectrum

def spect_load_1d(wkdir,time,species,xmin,xmax):
    """
    Inputs ----
    wkdir      : string, data directory
    time       : float,  [0,1] of the time_end 
    species    : int,    which species you want to plot
                         according to your own diagnostic in the Namelist
    xmin, xmax : float,  spatial selections, NOTE: in unit of de
    
    Outputs ---
    ekin       : float,  x-axis, energy(E) in MeV
    num        : float,  y-axis, E \frac{dN}{dE} \frac{1}{N}
                                 N  is the total number of particles
                                 dN is the number of particles in each energy range (dE)
                                 dE is the energy range
    simu_time  : float,  simulation time
    """
    S = happi.Open(wkdir)                        # No need to open the data again.
    dt = S.namelist.d_t                            # get simulation timestep
    ts = S.ParticleBinning(species).getTimesteps() # get all the timesteps available for this diag.
    iteration = int(ts[-1] * time)                 # choose the time you want by a factor of [0,1]*T_end
    simu_time = iteration * dt / wr                # convert it into real unit
    pb = S.ParticleBinning(species).get(iteration) # get the diag. data from the chosen timestep
    
    xx   = pb['x'] * de * 1e3                      # get position information and convert the unit to mm
    ekin = pb['ekin'] * 0.511                      # get energy information (E) and convert the unit to MeV
    data = pb['data'][0]                           # get the particle number information
    data_selected = data[(xx>xmin) & (xx<xmax),:]  # select particle number according to position
                                                   # for our case now, this doesn't make much difference.
    data_ekin = np.sum(data_selected,axis=0)       # dN, ?? can't remember...
    dE = ekin[1]-ekin[0]                           # dE, energy range
    total_weight = np.sum(data)                    # N, total number of the selected portion
    num = data_ekin * ekin / total_weight / dE     # E \frac{dN}{dE} \frac{1}{N}
    
    return ekin, num, simu_time

# Function for 2D streak map

def prepare_streak_2D(case, num, field_name, frame):
    
    """
    case: string, Smilei case
    num: '0', instant; '1', averaged 
    field_name: string, name of the field
    frame: float, velocity of the reference frame
    """
    var = case.Field(num,field_name,units=["mm","ns","cm^{-3}"]) # find the data
    data_var = var.getData()                           # get the data out
    data_var_2D = np.vstack(data_var)                  # transform it into 2D with (t,x)
    data_var_t = var.getTimes()                        # get the t-axis
    data_var_x = var.getAxis('x')                      # get the x-axis
    
    data_var_x_2D, data_var_t_2D = np.meshgrid(data_var_x, data_var_t, indexing='xy') # make the interpolation
    
    data_var_x_2D_frame = data_var_x_2D - frame * data_var_t_2D - Lx/2.               # change the reference frame (and the label) 
                                                                                      # note the unit used
    return data_var_x_2D, data_var_x_2D_frame, data_var_t_2D, data_var_2D

def Particle_trajectory(case, species, mass, condition):
    
    """
    case: string, Smilei case
    species: string, name of the traced particle
    mass: float, mass of the traced particle, used when calcuating momentum/energy etc.
    condition: string, Smilei formatted, to select particles satisfying a certain conditions.
    frame: float, velocity of the reference frame
    """
    
#     me = 9.1e-31
#     c  = 3.0e8
#     wr = S.namelist.w_r
#     de = c / wr 
    
    tp = case.TrackParticles(species,
                             select = condition,
                             axes=['x','px','py','pz','Ex','Ey','Bz','w'],
#                              units=['mm','ns'],                     # TODO: Use normalized unit during calculation
                             sort=True,
                            )
    data = tp.getData()                                           # get data
    time_tmp = data['times']                                      # get time, in unit of wpe^{-1}
    time_ns = np.linspace(0,tp.getTimes()[-1],time_tmp.size)      # transform time array into unit of ns.
    
    tp_px = data['px']                                            # extract all kinds of data
    tp_py = data['py']
    tp_pz = data['pz']
    tp_Ex = data['Ex']
    tp_Ey = data['Ey']
    tp_Bz = data['Bz']
    tp_x  = data['x']
    tp_weight = data['w']
    
#     tp_x_frame = tp_x * de * 1e3 - frame * time_ns              # change reference frame, do it later.
    
    tp_E  = (tp_px**2 + tp_py**2 + tp_pz**2)*(me*c)**2 / 2. / mass / qe / 1e6  # energy, in Unit of MeV
    tp_E = np.where(np.isfinite(tp_E),tp_E,0.)                                 # replace the inf/nan with 0.
    
    # find the particle with the highest energy at the last timestep 
    # (end of the simulation)
    [timestep, particle_number] = np.unravel_index(tp_E[-1].argmax(), tp_E.shape)
    
    tp_x_sub = tp_x[:,particle_number]
    tp_E_sub = tp_E[:,particle_number]
    
    return tp_x, tp_E, tp_px, tp_py, tp_pz, time_ns, tp_x_sub, tp_E_sub, tp_Ex, tp_Ey, tp_Bz, tp_weight, timestep, particle_number


In [15]:
# Fig.1, Start with phase space evolution

ps0 = S0.ParticleBinning(0,
                       units=Units, 
                       cmap='smilei_r',
                       vmin=1,vmax=4,
                       ymin=-1000, ymax=4000,
                       xmin=6.,xmax=10.,
                       data_log=True,
                      )
ps1 = S1.ParticleBinning(0,
                       units=Units, 
                       cmap='smilei_r',
                       vmin=1,vmax=4,
                       ymin=-1000, ymax=4000,
                       xmin=6.,xmax=10.,
                       data_log=True,
                      )

# ps2 = S2.ParticleBinning(0,
#                        units=Units, 
#                        cmap=newcmap,
#                        vmin=-1,vmax=3,
#                        ymin=-2000, ymax=4000,
# #                        xmin=6.,xmax=10.,
#                        data_log=True,
#                       )

happi.multiPlot(ps0,
                ps1,
#                 ps2,
                shape=[2,1],
                figsize=[8,6],
                skipAnimation=True,
                saveAs=wkdir[0]+'phase-space-B.pdf'
#                 movie=wkdir[0]+'phase-space.mp4',
#                 fps=15,
#                 dpi=300,
               )

In [9]:
ps2 = S2.ParticleBinning(0,
                       units=Units, 
                       cmap=newcmap,
                       vmin=-1,vmax=3,
                       ymin=-2000, ymax=4000,
#                        xmin=6.,xmax=10.,
                       data_log=True,
                      ).slide()

#0 - Number density of species # 0,4
    x from 0.0 to 2048.0 in 1500 steps 
    vx from -0.025 to 0.025 in 1500 steps 




/Users/yz/Documents/Smilei/happi/_Diagnostics/Diagnostic.py:850: RuntimeWarning: divide by zero encountered in log10
  return self._np.log10( self._vfactor*self._getDataAtTime(t) )


In [4]:
ekin0, num0, t_end0 = spect_load_1d(wkdir[0], 1.0, 1, 0.0, Lx)
ekin1, num1, t_end1 = spect_load_1d(wkdir[2], 1.0, 1, 0.0, Lx)
# ekin2, num2, t_end2 = spect_load_1d(wkdir[2], 1.0, 2, 0.0, Lx)
# ekin3, num3, t_end3 = spect_load_1d(wkdir[3], 1.0, 1, 0.0, Lx)

Loaded simulation '/Volumes/LaCie/Data_Smilei/SS/SS1_width200um/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics
Loaded simulation '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics


In [5]:
plt.rc('text', usetex=False)
mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 16#20
# mpl.rcParams['legend.fontname'] = 'Comic Sans MS'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 1.2

width  = 3.14 # single column, 8cm
height = width

fig, ax = plt.subplots()

# plt.ticklabel_format(axis='both', style='sci',useMathText=True)

fig.subplots_adjust(left=.2, bottom=.2, right=.9, top=.9)

ax.set_xscale('log')
ax.set_yscale('log')

exp = np.loadtxt('/Users/yz/Nextcloud/PROJECTS/Shock_2020/Shock_MS2020/Materials_For_Main/Fig_PIC_results_v2/spectres_av.txt')




# ax.plot(ekin0, num0, '-b',  label='2D, ppc=256,  Ly=128, dx=1.0de, time = {:.1f} ns'.format(t_end0*1e9), lw=1.0)
# ax.plot(ekin1, num1, '-r',  label='1D, ppc=1024, Ly=1,   dx=0.2de, time = {:.1f} ns'.format(t_end1*1e9), lw=1.0)
# ax.plot(ekin2, num2, '-g',  label='2D, ppc=256,  Ly=32,  dx=0.5de, time = {:.1f} ns'.format(t_end2*1e9), lw=1.0)

# ax.plot(ekin3, num3, '-k',  label='Bz=0', lw=1.0)
# ax.plot(ekin2, num2, '-g',  label='Bz=17T', lw=1.0)
ax.plot(ekin0, num0, '-k',  label='Sim. v = 1500 km/s', lw=1.0)
ax.plot(ekin1, num1, '--y', label='Sim. v =  500 km/s', lw=1.0)



# ax.plot(ekin2, num2, '-c',  label='time = {:.1f} ns'.format(t_end2*1e9), lw=1.0)
# ax.plot(ekin3, num3, '-m',  label='time = {:.1f} ns'.format(t_end3*1e9), lw=1.0)
# ax.plot(ekin4, num4, '-r',  label='time = {:.1f} ns'.format(t_end4*1e9), lw=1.0)

# def mypower(x, ampl, expo):
#     return ampl * x**expo

# ampl = 8.7197e-06
# expo = -2.23034800

# num6 = mypower(ekin4[90:1500], ampl, expo)
# ax.plot(ekin4[90:1500], num6, '--r',  label='fitted with p = {:.2f}'.format(expo), lw=2.0)



ax.set_xlabel(r'$E_{p}$ [MeV]',fontsize=10)
ax.set_ylabel(r'$E\frac{dN}{dE}\frac{1}{N}$',fontsize=10)

def func(x,kbT):
    edrift = 0.
    return x*np.sqrt(x)*np.exp(-(x-edrift)**2/kbT) 

index = (ekin0 < 0.08)*(ekin0 > 0.0005)
# popt, pcov = curve_fit(func, ekin0[index], num0[index])
# print(popt,pcov)
# y2 = func(ekin0[index], 100.0/1e6)
# ax.plot(ekin0[index], y2*4e3, 'r--',label='100eV')
y3 = func(ekin0[index], 200.0/1e6)
ax.plot(ekin0[index], y3*3e2, color='red',linestyle='dashdot',label='Thermal proton 200 eV')

ax.loglog(exp[0:-1:4,0],exp[0:-1:4,3]/1e14,color='cyan',linestyle='dotted',label='Exp. Noise Baseline')
ax.errorbar(exp[0:-1:4,0], exp[0:-1:4,1]/1e14, yerr=exp[0:-1:4,2]/1e14, 
            fmt='or',markersize=4,
            ecolor='b',elinewidth=1.0,
            capsize=2.0, capthick = 1.0,
            label='Exp. Data')

ax.tick_params(axis='both',which='major', direction='out',labelsize=8)
ax.minorticks_off()


ax.set_xlim([0.02, 0.10])
formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
# ax.xaxis.set_minor_formatter(formatter)
# ax.yaxis.set_major_formatter(formatter)
# ax.yaxis.set_minor_formatter(formatter)
ax.set_xticks([0.02, 0.05, 0.08, 0.1])
# ax.set_xticklabels([0.02, 0.05, 0.08, 0.10],fontsize=8)


ax.set_ylim([1e-5, 1])
ax.set_yticks([1, 0.01, 0.0001])
# ax.set_yticklabels([1, 0.01, 1],fontsize=8)

# ax.ticklabel_format(axis='both', style='sci',useMathText=True)


# ax.yaxis.set_major_formatter(plt.FormatStrFormatter(useMathText))

leg = ax.legend(loc='best', numpoints=1, fancybox=False, fontsize=6, frameon=False)
leg._legend_box.align = "right"
# ax.grid(which='both')
fig.set_size_inches(width, height)
fig.savefig('/Users/yz/Desktop/'+'Fig3_updated'+'.pdf',bbox_inches='tight',dpi=300)

In [6]:
# Fig.3, particle trajectories illustrating the acceleration mechanism

# prepare packages

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm, LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator

# prepare to convert to center-of-mass reference frame 
n1   = 2e18     # drifting plasma, number density
n2   = 1e18     # ambient plasma, number density
v1   = 1.5e6    # drifting velocity, m/s
v2   = 0.0
vcom = (n1*v1 + n2*v2) / (n1+n2) # center-of-mass reference frame, m/s
vcom_mm_ns = vcom * 1e-6         # in unit of mm/ns

# prepare to convert to discontinuity reference frame
vcd = 0.86 # mm/ns, read from the x-t plot.

# # get data for 2D streak map
# Bz_x_0, Bz_x, Bz_t, Bz = prepare_streak_2D(S, 0, 'Bz', vcom_mm_ns)
# ni_x_0, ni_x, ni_t, ni = prepare_streak_2D(S, 0, 'Rho_ion1+Rho_ion2', vcom_mm_ns)

Bz_x_1, Bz_x_cd,  Bz_t_cd,  Bz_cd  = prepare_streak_2D(S1, 0, 'Bz', vcd)
ni_x_1, ni_x_cd,  ni_t_cd,  ni_cd  = prepare_streak_2D(S1, 0, 'Rho_ion1+Rho_ion2', vcd)
Ex_x_2, Ex_x_cd2, Ex_t_cd2, Ex_cd2 = prepare_streak_2D(S1, 1, 'Ex', vcd)
Ey_x_2, Ey_x_cd2, Ey_t_cd2, Ey_cd2 = prepare_streak_2D(S1, 1, 'Ey', vcd)
Bz_x_2, Bz_x_cd2, Bz_t_cd2, Bz_cd2 = prepare_streak_2D(S1, 1, 'Bz', vcd)

In [36]:
def prepare_streak_2D(case, num, field_name, frame):
    
    """
    case: string, Smilei case
    num: '0', instant; '1', averaged 
    field_name: string, name of the field
    frame: float, velocity of the reference frame
    """
    var = case.Field(num,field_name,units=["mm","ns","cm^{-3}","T","V/m"]) # find the data
    data_var = var.getData()                           # get the data out
    data_var_2D = np.vstack(data_var)                  # transform it into 2D with (t,x)
    data_var_t = var.getTimes()                        # get the t-axis
    data_var_x = var.getAxis('x')                      # get the x-axis
    
    data_var_x_2D, data_var_t_2D = np.meshgrid(data_var_x, data_var_t, indexing='xy') # make the interpolation
    
    data_var_x_2D_frame = data_var_x_2D - frame * data_var_t_2D - Lx/2.               # change the reference frame (and the label) 
                                                                                      # note the unit used
    return data_var_x_2D, data_var_x_2D_frame, data_var_t_2D, data_var_2D

In [37]:
ni_x_1, ni_x_cd,  ni_t_cd,  ni_cd  = prepare_streak_2D(S1, 0, 'Rho_ion1+Rho_ion2', vcd)
Ex_x_2, Ex_x_cd2, Ex_t_cd2, Ex_cd2 = prepare_streak_2D(S1, 1, 'Ex', vcd)
Ey_x_2, Ey_x_cd2, Ey_t_cd2, Ey_cd2 = prepare_streak_2D(S1, 1, 'Ey', vcd)
Bz_x_2, Bz_x_cd2, Bz_t_cd2, Bz_cd2 = prepare_streak_2D(S1, 1, 'Bz', vcd)

In [66]:
width = 1.6*2*2
height = width*1.2

time_to_plot = 1.5 # in ns
t_end = S1.namelist.t_end  / wr * 1e9 # in ns
time_cell = int(time_to_plot * Bz_x_cd2[:,0].size / t_end)

# kw = {'height_ratios':[13,4]}
fig, (ax0,ax1,ax2,ax3,ax4) = plt.subplots(5,1, sharex=True)

ax0.plot(ni_x_cd[time_cell,:] ,ni_cd[time_cell,:],'-k',label='ni',lw=1.0)
ax0.set_ylabel('ni [cm^-3]')
ax0.set_ylim([0,5e18])
ax1.plot(Bz_x_cd2[time_cell,:],Bz_cd2[time_cell,:],'-r',label='Bz',lw=1.0)
ax1.set_xlim([0.4,1.2])
ax1.set_ylabel('Bz [T]')
ax2.plot(Ey_x_cd2[time_cell,:],Ey_cd2[time_cell,:],'-b',label='Ey',lw=1.0)
ax2.set_ylim([-1e7,9e7])
ax2.set_ylabel('Ey [V/m]')
ax3.plot(Ex_x_cd2[time_cell,:],Ex_cd2[time_cell,:],'-m',label='Ex',lw=1.0)
ax3.set_ylim([-5e8,5e8])
ax3.set_ylabel('Ex [V/m]')
# ax3.set_xlabel('x [mm]')

# find the same timestep for phase-space data
ts = S1.ParticleBinning(0).getTimesteps()
# it = int(ts[-1]*time_to_plot/t_end)    # here, not exactly the timestep, try to fix.
it = 845000                              # this value, find the closest available value to the above equation.
ps = S1.ParticleBinning(0).get(it)
xx = ps['x']*de*1e3 - time_to_plot*vcd - Lx/2.   # convert unit to mm
vx = ps['vx']*c  # convert unit to m/s
da = np.log10(ps['data'][0])
xmin = 0.4
xmax = 1.2
xcmi = int((xmin-xx.min())*xx.size/(xx.max()-xx.min()))
xcma = int((xmax-xx.max())*xx.size/(xx.max()-xx.min()))
vmin = -300000.0
vmax = 2300000.0
vcmi = int((vmin-vx.min())*vx.size/(vx.max()-vx.min()))
vcma = int((vmax-vx.min())*vx.size/(vx.max()-vx.min()))
ax4.imshow(da[xcmi:xcma,vcmi:vcma].T,
          extent=[xmin,xmax,vmin,vmax],
          origin='lower',
          aspect='auto',
          cmap='smilei')
ax4.set_ylabel('Vx [m/s]')
ax4.set_xlabel('x [mm]')
# ax1.set_yticks([0, 1])



fig.set_size_inches(width, height)

# plt.show()
fig.savefig('/Users/yz/Desktop/'+'field_structure_SI'+'.pdf',bbox_inches='tight',dpi=300)




<ipython-input-66-41067c6d7b40>:32: RuntimeWarning: divide by zero encountered in log10
  da = np.log10(ps['data'][0])


In [35]:
# Draw the figure of x-Bz-Ex-Vx

width = 1.6
height = width

time_to_plot = 1.5 # in ns
t_end = S1.namelist.t_end  / wr * 1e9 # in ns
time_cell = int(time_to_plot * Bz_x_cd2[:,0].size / t_end)

# kw = {'height_ratios':[13,4]}
fig, (ax0,ax1) = plt.subplots(2,1, sharex=True)

ax0.plot(Bz_x_cd2[time_cell,:],Bz_cd2[time_cell,:]/0.25,'-r',label='Bz',lw=0.5)
ax0.plot(Ey_x_cd2[time_cell,:],Ey_cd2[time_cell,:]/0.001,'--b',label='Ey',lw=0.5)
ax0.plot(ni_x_cd[time_cell,:] ,ni_cd[time_cell,:] /4.2e18 ,'--k',label='ni',lw=0.5)
ax0.plot(Ex_x_cd2[time_cell,:],Ex_cd2[time_cell,:]/0.005,'-m',label='Ex',lw=0.5)
ax0.set_ylim([-1,1])
ax0.set_ylabel('A. U.')
ax0.set_yticks([-1, 0, 1])
ax0.legend(fontsize=3,frameon=False)
# ax0.grid(linestyle='--', linewidth=0.5, alpha=0.5)


# find the same timestep for phase-space data
ts = S1.ParticleBinning(0).getTimesteps()
# it = int(ts[-1]*time_to_plot/t_end)    # here, not exactly the timestep, try to fix.
it = 845000                              # this value, find the closest available value to the above equation.

# get the acorresponding data out
ps = S1.ParticleBinning(0).get(it)
xx = ps['x']*de*1e3 - time_to_plot*vcd - Lx/2.   # convert unit to mm
vx = ps['vx']*c*1e-3  # convert unit to km/s
da = np.log10(ps['data'][0])

# xmin = 6.4 - time_to_plot*vcd - Lx/2.
# xmax = 7.0 - time_to_plot*vcd - Lx/2.
xmin = 0.4
xmax = 1.2
xcmi = int((xmin-xx.min())*xx.size/(xx.max()-xx.min()))
xcma = int((xmax-xx.max())*xx.size/(xx.max()-xx.min()))

vmin = -300.0
vmax = 2300.0
vcmi = int((vmin-vx.min())*vx.size/(vx.max()-vx.min()))
vcma = int((vmax-vx.min())*vx.size/(vx.max()-vx.min()))
ax1.imshow(da[xcmi:xcma,vcmi:vcma].T/vmax,
          extent=[xmin,xmax,-0.2,1],
          origin='lower',
          aspect='auto',
          cmap='smilei')
ax1.set_ylabel(r'$v_x$ [A. U.]')
ax1.set_yticks([0, 1])
# ax1.grid(linestyle='--', linewidth=0.5, alpha=0.5)

ax1.set_xlabel('x [mm]')
# fig.suptitle('time = {:.2f} ns'.format(time_to_plot))
fig.set_size_inches(width, height)
fig.savefig('/Users/yz/Desktop/'+'field_structure2'+'.pdf',bbox_inches='tight',dpi=300)


<ipython-input-35-11d46efa7187>:33: RuntimeWarning: divide by zero encountered in log10
  da = np.log10(ps['data'][0])


In [21]:
plt.rc('text', usetex=False)
mpl.rcParams['font.family'] = 'Arial'
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 16#20
# mpl.rcParams['legend.fontname'] = 'Comic Sans MS'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 1.2

width  = 3.14 # single column, 8cm
height = width
fig, ax = plt.subplots()

plt.ticklabel_format(axis='both', style='sci',useMathText=True)

fig.subplots_adjust(left=.2, bottom=.2, right=.9, top=.9)

ax.set_xscale('log')
ax.set_yscale('log')

exp = np.loadtxt('/Users/yz/Nextcloud/PROJECTS/Shock_2020/Shock_MS2020/Materials_For_Main/Fig_PIC_results_v2/spectres_av.txt')
ax.errorbar(exp[0:-1:4,0], exp[0:-1:4,1]/1e14, yerr=exp[0:-1:4,2]/1e14, 
            fmt='or',markersize=4,
            ecolor='b',elinewidth=1.0,
            capsize=2.0, capthick = 1.0,
            
            label='Exp. Data')

# ax.plot(ekin3, num3, '-k',  label='Bz=0', lw=1.0)
ax.plot(ekin0, num0, '-k',  label='Simulation', lw=1.0)
# ax.plot(ekin2, num2, '-g',  label='Bz=17T', lw=1.0)
# ax.plot(ekin0, num0, '-y',  label='Shock Width=di~200um', lw=1.0)


ax.tick_params(axis='both',which='major', direction='out',labelsize=8)
ax.minorticks_off()

ax.set_xlabel(r'$E_{p}$ [MeV]',fontsize=10)
ax.set_ylabel(r'$E\frac{dN}{dE}\frac{1}{N}$',fontsize=10)


ax.set_xlim([0.02, 0.08])
formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
ax.set_xticks([0.02, 0.05, 0.08])
# ax.set_xticklabels([0.01, 0.05, 0.10],fontsize=8)


ax.set_ylim([1e-5, 1])
ax.set_yticks([1, 0.01, 0.0001])

def func(x,kbT):
    edrift = 0.
    return x*np.sqrt(x)*np.exp(-(x-edrift)**2/kbT) 

index = (ekin0 < 0.08)*(ekin0 > 0.0005)
# popt, pcov = curve_fit(func, ekin0[index], num0[index])
# print(popt,pcov)
y2 = func(ekin0[index], 100.0/1e6)
ax.plot(ekin0[index], y2*4e3, 'r--',label='100eV')
y3 = func(ekin0[index], 200.0/1e6)
ax.plot(ekin0[index], y3*3e2, 'c--',label='200eV')

leg = ax.legend(loc='best', numpoints=1, fancybox=False, fontsize=6, frameon=False)
leg._legend_box.align = "right"
# ax.grid(which='both')
fig.set_size_inches(width, height)
plt.show()
fig.savefig('/Users/yz/Desktop/'+'thermal_spe'+'.pdf',bbox_inches='tight',dpi=300)

In [23]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x,kbT):
    edrift = 0.
    return x*np.sqrt(x)*np.exp(-(x-edrift)**2/kbT) 

index = (ekin0 < 0.02)*(ekin0 > 0.001)
# popt, pcov = curve_fit(func, ekin0[index], num0[index])
# print(popt,pcov)
y2 = func(ekin0[index], 100.0/1e6)
ax.plot(ekin0[index], y2, 'r--')

In [87]:
np.save('/Users/yz/Desktop/ene.npy',ekin0)


In [88]:
np.save('/Users/yz/Desktop/num.npy',num0)

In [ ]:
def ori_spect_load_1d(wkdir,time,species,xmin,xmax):
    """
    Inputs ----
    wkdir      : string, data directory
    time       : float,  [0,1] of the time_end 
    species    : int,    which species you want to plot
                         according to your own diagnostic in the Namelist
    xmin, xmax : float,  spatial selections, NOTE: in unit of de
    
    Outputs ---
    ekin       : float,  x-axis, energy(E) in MeV
    num        : float,  y-axis, E \frac{dN}{dE} \frac{1}{N}
                                 N  is the total number of particles
                                 dN is the number of particles in each energy range (dE)
                                 dE is the energy range
    simu_time  : float,  simulation time
    """
    S = happi.Open(wkdir)                        # No need to open the data again.
    dt = S.namelist.d_t                            # get simulation timestep
    ts = S.ParticleBinning(species).getTimesteps() # get all the timesteps available for this diag.
    iteration = int(ts[-1] * time)                 # choose the time you want by a factor of [0,1]*T_end
    simu_time = iteration * dt / wr                # convert it into real unit
    pb = S.ParticleBinning(species).get(iteration) # get the diag. data from the chosen timestep
    
    xx   = pb['x'] * de * 1e3                      # get position information and convert the unit to mm
    ekin = pb['ekin'] * 0.511                      # get energy information (E) and convert the unit to MeV
    data = pb['data'][0]                           # get the particle number information
    data_selected = data[(xx>xmin) & (xx<xmax),:]  # select particle number according to position
                                                   # for our case now, this doesn't make much difference.
    num = data
#     data_ekin = np.sum(data_selected,axis=0)       # dN, ?? can't remember...
#     dE = ekin[1]-ekin[0]                           # dE, energy range
#     total_weight = np.sum(data)                    # N, total number of the selected portion
#     num = data_ekin * ekin / total_weight / dE     # E \frac{dN}{dE} \frac{1}{N}
    
    return ekin, num, simu_time

In [9]:
def test_Particle_trajectory(case, species, mass):
    
    """
    case: string, Smilei case
    species: string, name of the traced particle
    mass: float, mass of the traced particle, used when calcuating momentum/energy etc.
    condition: string, Smilei formatted, to select particles satisfying a certain conditions.
    frame: float, velocity of the reference frame
    """
    
#     me = 9.1e-31
#     c  = 3.0e8
#     wr = S.namelist.w_r
#     de = c / wr 
    
    tp = case.TrackParticles(species,
#                              select = condition,
                             axes=['x','px','py','pz','Ex','Ey','Bz','w'],
#                              units=['mm','ns'],                     # TODO: Use normalized unit during calculation
                             sort=True,
                            )
    data = tp.getData()                                           # get data
    time_tmp = data['times']                                      # get time, in unit of wpe^{-1}
    time_ns = np.linspace(0,tp.getTimes()[-1],time_tmp.size)      # transform time array into unit of ns.
    
    tp_px = data['px']                                            # extract all kinds of data
    tp_py = data['py']
    tp_pz = data['pz']
    tp_Ex = data['Ex']
    tp_Ey = data['Ey']
    tp_Bz = data['Bz']
    tp_x  = data['x']
    tp_weight = data['w']
    
#     tp_x_frame = tp_x * de * 1e3 - frame * time_ns              # change reference frame, do it later.
    
    tp_E  = (tp_px**2 + tp_py**2 + tp_pz**2)*(me*c)**2 / 2. / mass / qe / 1e6  # energy, in Unit of MeV
    tp_E = np.where(np.isfinite(tp_E),tp_E,0.)                                 # replace the inf/nan with 0.
    
    # find the particle with the highest energy at the last timestep 
    # (end of the simulation)
    [timestep, particle_number] = np.unravel_index(tp_E[-1].argmax(), tp_E.shape)
    
    tp_x_sub = tp_x[:,particle_number]
    tp_E_sub = tp_E[:,particle_number]
    
    return tp_E, tp_px, tp_py, tp_pz, tp_weight


In [11]:
ene1, tp_px1, tp_py1, tp_pz1, wgt1 = test_Particle_trajectory(S1, 'ion1t', mp)

Selecting particles ... (this may take a while)
Removing dead particles ...
Kept 5120 particles
Loading data ...
   axis: x
   axis: px
   axis: py
   axis: pz
   axis: Ex
   axis: Ey
   axis: Bz
   axis: w
Process broken lines ...
... done


In [12]:
ene2, tp_px2, tp_py2, tp_pz2,wgt2 = test_Particle_trajectory(S1, 'ion2t', mp)

Selecting particles ... (this may take a while)
Removing dead particles ...
Kept 5120 particles
Loading data ...
   axis: x
   axis: px
   axis: py
   axis: pz
   axis: Ex
   axis: Ey
   axis: Bz
   axis: w
Process broken lines ...
... done


In [ ]:
plt.hist(ene1)

In [7]:
np.save('/Users/yz/Desktop/ene1.npy',ene1)

In [13]:
np.save('/Users/yz/Desktop/px1.npy',tp_px1)
np.save('/Users/yz/Desktop/py1.npy',tp_py1)
np.save('/Users/yz/Desktop/pz1.npy',tp_pz1)